In [1]:
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
tf.__version__

'2.2.0-dev20200317'

In [3]:
loss_metric = tf.keras.metrics.Mean()


## Sampling

In [4]:
class Sampling(layers.Layer):
  """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

## Encoder

In [5]:
class Encoder(layers.Layer):
  """Maps MNIST digits to a triplet (z_mean, z_log_var, z)."""

  def __init__(self,
               latent_dim=32,
               intermediate_dim=64,
               name='encoder',
               **kwargs):
    super(Encoder, self).__init__(name=name, **kwargs)
    self.dense_proj = layers.Dense(intermediate_dim, activation='relu')
    self.dense_mean = layers.Dense(latent_dim)
    self.dense_log_var = layers.Dense(latent_dim)
    self.sampling = Sampling()

  def call(self, inputs):
    x = self.dense_proj(inputs)
    z_mean = self.dense_mean(x)
    z_log_var = self.dense_log_var(x)
    z = self.sampling((z_mean, z_log_var))
    return z_mean, z_log_var, z


## Decoder

In [6]:
class Decoder(layers.Layer):
  """Converts z, the encoded digit vector, back into a readable digit."""

  def __init__(self,
               original_dim,
               intermediate_dim=64,
               name='decoder',
               **kwargs):
    super(Decoder, self).__init__(name=name, **kwargs)
    self.dense_proj = layers.Dense(intermediate_dim, activation='relu')
    self.dense_output = layers.Dense(original_dim, activation='sigmoid')

  def call(self, inputs):
    x = self.dense_proj(inputs)
    return self.dense_output(x)

## Variational auto encoder

In [7]:
class VariationalAutoEncoder(tf.keras.Model):
  """Combines the encoder and decoder into an end-to-end model for training."""

  def __init__(self,
               original_dim,
               intermediate_dim=64,
               latent_dim=32,
               name='autoencoder',
               **kwargs):
    super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
    self.original_dim = original_dim
    self.encoder = Encoder(latent_dim=latent_dim,
                           intermediate_dim=intermediate_dim)
    self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

  def call(self, inputs):
    z_mean, z_log_var, z = self.encoder(inputs)
    reconstructed = self.decoder(z)
    # Add KL divergence regularization loss.
    kl_loss = - 0.5 * tf.reduce_mean(
        z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
    self.add_loss(kl_loss)
    return reconstructed

## Training

In [8]:
original_dim = 784
vae = VariationalAutoEncoder(original_dim, 64, 32)

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)


In [10]:
mse_loss_fn = tf.keras.losses.MeanSquaredError()


In [11]:
(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255

train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)


In [14]:
epochs = 10


In [15]:
# Iterate over epochs.
for epoch in range(epochs):
  print('Start of epoch %d' % (epoch,))

  # Iterate over the batches of the dataset.
  for step, x_batch_train in enumerate(train_dataset):
    with tf.GradientTape() as tape:
      reconstructed = vae(x_batch_train)
      # Compute reconstruction loss
      loss = mse_loss_fn(x_batch_train, reconstructed)
      loss += sum(vae.losses)  # Add KLD regularization loss

    grads = tape.gradient(loss, vae.trainable_weights)
    optimizer.apply_gradients(zip(grads, vae.trainable_weights))

    loss_metric(loss)

    if step % 100 == 0:
      print('step %s: mean loss = %s' % (step, loss_metric.result()))


Start of epoch 0
step 0: mean loss = tf.Tensor(0.07463287, shape=(), dtype=float32)
step 100: mean loss = tf.Tensor(0.073982626, shape=(), dtype=float32)
step 200: mean loss = tf.Tensor(0.07347118, shape=(), dtype=float32)
step 300: mean loss = tf.Tensor(0.07301432, shape=(), dtype=float32)
step 400: mean loss = tf.Tensor(0.072680764, shape=(), dtype=float32)
step 500: mean loss = tf.Tensor(0.072278194, shape=(), dtype=float32)
step 600: mean loss = tf.Tensor(0.071991295, shape=(), dtype=float32)
step 700: mean loss = tf.Tensor(0.071701154, shape=(), dtype=float32)
step 800: mean loss = tf.Tensor(0.07147088, shape=(), dtype=float32)
step 900: mean loss = tf.Tensor(0.07119507, shape=(), dtype=float32)
Start of epoch 1
step 0: mean loss = tf.Tensor(0.07113065, shape=(), dtype=float32)
step 100: mean loss = tf.Tensor(0.0709553, shape=(), dtype=float32)
step 200: mean loss = tf.Tensor(0.07082657, shape=(), dtype=float32)
step 300: mean loss = tf.Tensor(0.07067196, shape=(), dtype=float32)
